In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

In [10]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Create a directory if not exists
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
save_dir = 'images'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [4]:
# Hyper-parameters
image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

# MNIST dataset
dataset = torchvision.datasets.MNIST(root=data_dir,
                                     train=True,
                                     transform=transforms.ToTensor(),
                                     download=True)

In [5]:
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [6]:
# VAE model
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)

    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)

    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var / 2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var


In [7]:
model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Start training
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # Forward pass
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)

        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

        # Backprop and optimize
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            print("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}"
                  .format(epoch + 1, num_epochs, i + 1, len(data_loader), reconst_loss.item(), kl_div.item()))

    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join(save_dir, 'sampled-{}.png'.format(epoch + 1)))

        # Save the reconstructed images
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(save_dir, 'reconst-{}.png'.format(epoch + 1)))

Epoch[1/15], Step [10/3750], Reconst Loss: 1533.8453, KL Div: 401.9591
Epoch[1/15], Step [20/3750], Reconst Loss: 1460.3625, KL Div: 432.3881
Epoch[1/15], Step [30/3750], Reconst Loss: 1386.8225, KL Div: 409.1543
Epoch[1/15], Step [40/3750], Reconst Loss: 1212.4360, KL Div: 366.2297
Epoch[1/15], Step [50/3750], Reconst Loss: 1459.8016, KL Div: 395.0373
Epoch[1/15], Step [60/3750], Reconst Loss: 1488.9297, KL Div: 422.2281
Epoch[1/15], Step [70/3750], Reconst Loss: 1455.2010, KL Div: 410.7771
Epoch[1/15], Step [80/3750], Reconst Loss: 1323.4392, KL Div: 418.4683
Epoch[1/15], Step [90/3750], Reconst Loss: 1462.0778, KL Div: 382.4022
Epoch[1/15], Step [100/3750], Reconst Loss: 1509.4679, KL Div: 391.7671
Epoch[1/15], Step [110/3750], Reconst Loss: 1462.1409, KL Div: 404.0363
Epoch[1/15], Step [120/3750], Reconst Loss: 1262.4585, KL Div: 376.9858
Epoch[1/15], Step [130/3750], Reconst Loss: 1506.0088, KL Div: 430.3894
Epoch[1/15], Step [140/3750], Reconst Loss: 1375.8207, KL Div: 391.2036
E

Epoch[1/15], Step [1180/3750], Reconst Loss: 1182.3004, KL Div: 377.6833
Epoch[1/15], Step [1190/3750], Reconst Loss: 1357.7819, KL Div: 392.9539
Epoch[1/15], Step [1200/3750], Reconst Loss: 1183.5491, KL Div: 365.5579
Epoch[1/15], Step [1210/3750], Reconst Loss: 1488.1531, KL Div: 387.2328
Epoch[1/15], Step [1220/3750], Reconst Loss: 1393.7571, KL Div: 391.7798
Epoch[1/15], Step [1230/3750], Reconst Loss: 1460.3469, KL Div: 397.6514
Epoch[1/15], Step [1240/3750], Reconst Loss: 1464.9813, KL Div: 417.1177
Epoch[1/15], Step [1250/3750], Reconst Loss: 1443.6060, KL Div: 381.0266
Epoch[1/15], Step [1260/3750], Reconst Loss: 1540.7505, KL Div: 418.3638
Epoch[1/15], Step [1270/3750], Reconst Loss: 1356.4738, KL Div: 404.2671
Epoch[1/15], Step [1280/3750], Reconst Loss: 1186.8577, KL Div: 396.0219
Epoch[1/15], Step [1290/3750], Reconst Loss: 1440.1248, KL Div: 410.6388
Epoch[1/15], Step [1300/3750], Reconst Loss: 1321.2761, KL Div: 379.2541
Epoch[1/15], Step [1310/3750], Reconst Loss: 1401.8

Epoch[1/15], Step [2320/3750], Reconst Loss: 1541.9468, KL Div: 427.0160
Epoch[1/15], Step [2330/3750], Reconst Loss: 1392.1597, KL Div: 414.8062
Epoch[1/15], Step [2340/3750], Reconst Loss: 1526.9633, KL Div: 407.0724
Epoch[1/15], Step [2350/3750], Reconst Loss: 1288.5706, KL Div: 408.9710
Epoch[1/15], Step [2360/3750], Reconst Loss: 1351.2310, KL Div: 378.4028
Epoch[1/15], Step [2370/3750], Reconst Loss: 1212.2329, KL Div: 380.7574
Epoch[1/15], Step [2380/3750], Reconst Loss: 1365.9634, KL Div: 398.6123
Epoch[1/15], Step [2390/3750], Reconst Loss: 1277.4669, KL Div: 390.2568
Epoch[1/15], Step [2400/3750], Reconst Loss: 1359.4427, KL Div: 394.9953
Epoch[1/15], Step [2410/3750], Reconst Loss: 1384.8093, KL Div: 395.0423
Epoch[1/15], Step [2420/3750], Reconst Loss: 1196.2845, KL Div: 390.2516
Epoch[1/15], Step [2430/3750], Reconst Loss: 1365.4373, KL Div: 419.4198
Epoch[1/15], Step [2440/3750], Reconst Loss: 1291.0938, KL Div: 383.4675
Epoch[1/15], Step [2450/3750], Reconst Loss: 1258.0

Epoch[1/15], Step [3470/3750], Reconst Loss: 1471.7443, KL Div: 388.9315
Epoch[1/15], Step [3480/3750], Reconst Loss: 1289.4944, KL Div: 350.7173
Epoch[1/15], Step [3490/3750], Reconst Loss: 1299.7373, KL Div: 402.2343
Epoch[1/15], Step [3500/3750], Reconst Loss: 1290.7627, KL Div: 381.0642
Epoch[1/15], Step [3510/3750], Reconst Loss: 1483.2747, KL Div: 407.8805
Epoch[1/15], Step [3520/3750], Reconst Loss: 1269.2031, KL Div: 375.3132
Epoch[1/15], Step [3530/3750], Reconst Loss: 1333.6357, KL Div: 415.1691
Epoch[1/15], Step [3540/3750], Reconst Loss: 1157.0952, KL Div: 381.4187
Epoch[1/15], Step [3550/3750], Reconst Loss: 1330.6071, KL Div: 348.8560
Epoch[1/15], Step [3560/3750], Reconst Loss: 1471.7135, KL Div: 426.8888
Epoch[1/15], Step [3570/3750], Reconst Loss: 1278.1665, KL Div: 401.4596
Epoch[1/15], Step [3580/3750], Reconst Loss: 1365.6565, KL Div: 374.0257
Epoch[1/15], Step [3590/3750], Reconst Loss: 1279.6099, KL Div: 383.6151
Epoch[1/15], Step [3600/3750], Reconst Loss: 1357.6

Epoch[2/15], Step [870/3750], Reconst Loss: 1421.8877, KL Div: 437.4772
Epoch[2/15], Step [880/3750], Reconst Loss: 1422.2084, KL Div: 410.2214
Epoch[2/15], Step [890/3750], Reconst Loss: 1371.5796, KL Div: 384.4930
Epoch[2/15], Step [900/3750], Reconst Loss: 1239.4048, KL Div: 361.2291
Epoch[2/15], Step [910/3750], Reconst Loss: 1451.3663, KL Div: 417.5812
Epoch[2/15], Step [920/3750], Reconst Loss: 1369.4438, KL Div: 413.3073
Epoch[2/15], Step [930/3750], Reconst Loss: 1336.2552, KL Div: 416.0056
Epoch[2/15], Step [940/3750], Reconst Loss: 1348.5760, KL Div: 391.9192
Epoch[2/15], Step [950/3750], Reconst Loss: 1269.3385, KL Div: 385.3804
Epoch[2/15], Step [960/3750], Reconst Loss: 1418.4797, KL Div: 406.8695
Epoch[2/15], Step [970/3750], Reconst Loss: 1453.7709, KL Div: 411.8159
Epoch[2/15], Step [980/3750], Reconst Loss: 1516.4050, KL Div: 403.9073
Epoch[2/15], Step [990/3750], Reconst Loss: 1261.7910, KL Div: 409.7096
Epoch[2/15], Step [1000/3750], Reconst Loss: 1346.9387, KL Div: 

...

Epoch[14/15], Step [1930/3750], Reconst Loss: 1389.0833, KL Div: 454.0654
Epoch[14/15], Step [1940/3750], Reconst Loss: 1228.1357, KL Div: 373.9400
Epoch[14/15], Step [1950/3750], Reconst Loss: 1091.6134, KL Div: 339.1904
Epoch[14/15], Step [1960/3750], Reconst Loss: 1310.0237, KL Div: 396.1155
Epoch[14/15], Step [1970/3750], Reconst Loss: 1265.5616, KL Div: 413.4220
Epoch[14/15], Step [1980/3750], Reconst Loss: 1103.0876, KL Div: 375.4519
Epoch[14/15], Step [1990/3750], Reconst Loss: 1225.8206, KL Div: 426.4113
Epoch[14/15], Step [2000/3750], Reconst Loss: 1352.8363, KL Div: 410.4169
Epoch[14/15], Step [2010/3750], Reconst Loss: 1318.1792, KL Div: 384.6297
Epoch[14/15], Step [2020/3750], Reconst Loss: 1319.0046, KL Div: 372.4624
Epoch[14/15], Step [2030/3750], Reconst Loss: 1211.8595, KL Div: 387.1728
Epoch[14/15], Step [2040/3750], Reconst Loss: 1240.4141, KL Div: 408.9602
Epoch[14/15], Step [2050/3750], Reconst Loss: 1381.9316, KL Div: 408.2188
Epoch[14/15], Step [2060/3750], Recons

Epoch[14/15], Step [3080/3750], Reconst Loss: 1344.2870, KL Div: 418.3229
Epoch[14/15], Step [3090/3750], Reconst Loss: 1217.6259, KL Div: 373.9654
Epoch[14/15], Step [3100/3750], Reconst Loss: 1149.0139, KL Div: 371.8190
Epoch[14/15], Step [3110/3750], Reconst Loss: 1385.2218, KL Div: 461.3250
Epoch[14/15], Step [3120/3750], Reconst Loss: 1313.0553, KL Div: 410.9343
Epoch[14/15], Step [3130/3750], Reconst Loss: 1200.2498, KL Div: 364.4393
Epoch[14/15], Step [3140/3750], Reconst Loss: 1293.2314, KL Div: 411.1840
Epoch[14/15], Step [3150/3750], Reconst Loss: 1264.4741, KL Div: 386.4153
Epoch[14/15], Step [3160/3750], Reconst Loss: 1281.6962, KL Div: 403.3272
Epoch[14/15], Step [3170/3750], Reconst Loss: 1165.4103, KL Div: 413.5620
Epoch[14/15], Step [3180/3750], Reconst Loss: 1224.2297, KL Div: 393.7796
Epoch[14/15], Step [3190/3750], Reconst Loss: 1356.3013, KL Div: 383.1028
Epoch[14/15], Step [3200/3750], Reconst Loss: 1289.2859, KL Div: 403.6988
Epoch[14/15], Step [3210/3750], Recons

Epoch[15/15], Step [480/3750], Reconst Loss: 1334.5441, KL Div: 399.1233
Epoch[15/15], Step [490/3750], Reconst Loss: 1342.4683, KL Div: 414.2548
Epoch[15/15], Step [500/3750], Reconst Loss: 1234.9390, KL Div: 380.3334
Epoch[15/15], Step [510/3750], Reconst Loss: 1343.8923, KL Div: 400.9577
Epoch[15/15], Step [520/3750], Reconst Loss: 1424.9769, KL Div: 421.3101
Epoch[15/15], Step [530/3750], Reconst Loss: 1174.8384, KL Div: 410.5812
Epoch[15/15], Step [540/3750], Reconst Loss: 1283.9706, KL Div: 399.7683
Epoch[15/15], Step [550/3750], Reconst Loss: 1372.1633, KL Div: 418.4919
Epoch[15/15], Step [560/3750], Reconst Loss: 1379.9005, KL Div: 385.4843
Epoch[15/15], Step [570/3750], Reconst Loss: 1263.3246, KL Div: 430.5536
Epoch[15/15], Step [580/3750], Reconst Loss: 1330.5764, KL Div: 407.2859
Epoch[15/15], Step [590/3750], Reconst Loss: 1207.4083, KL Div: 400.9817
Epoch[15/15], Step [600/3750], Reconst Loss: 1228.9443, KL Div: 385.8218
Epoch[15/15], Step [610/3750], Reconst Loss: 1270.7

Epoch[15/15], Step [1630/3750], Reconst Loss: 1293.4319, KL Div: 413.1407
Epoch[15/15], Step [1640/3750], Reconst Loss: 1213.7153, KL Div: 384.5074
Epoch[15/15], Step [1650/3750], Reconst Loss: 1230.7672, KL Div: 391.8916
Epoch[15/15], Step [1660/3750], Reconst Loss: 1420.9144, KL Div: 420.1404
Epoch[15/15], Step [1670/3750], Reconst Loss: 1229.9310, KL Div: 396.5425
Epoch[15/15], Step [1680/3750], Reconst Loss: 1169.6169, KL Div: 388.9551
Epoch[15/15], Step [1690/3750], Reconst Loss: 1352.6025, KL Div: 387.7061
Epoch[15/15], Step [1700/3750], Reconst Loss: 1313.1963, KL Div: 405.3571
Epoch[15/15], Step [1710/3750], Reconst Loss: 1299.3319, KL Div: 416.7928
Epoch[15/15], Step [1720/3750], Reconst Loss: 1322.9257, KL Div: 404.9638
Epoch[15/15], Step [1730/3750], Reconst Loss: 1145.0764, KL Div: 372.7062
Epoch[15/15], Step [1740/3750], Reconst Loss: 1172.6089, KL Div: 374.2015
Epoch[15/15], Step [1750/3750], Reconst Loss: 1317.2513, KL Div: 419.9326
Epoch[15/15], Step [1760/3750], Recons

Epoch[15/15], Step [2780/3750], Reconst Loss: 1324.1250, KL Div: 413.0642
Epoch[15/15], Step [2790/3750], Reconst Loss: 1283.6028, KL Div: 424.5361
Epoch[15/15], Step [2800/3750], Reconst Loss: 1243.3884, KL Div: 371.8993
Epoch[15/15], Step [2810/3750], Reconst Loss: 1273.5955, KL Div: 411.2302
Epoch[15/15], Step [2820/3750], Reconst Loss: 1199.5869, KL Div: 383.2377
Epoch[15/15], Step [2830/3750], Reconst Loss: 1228.4436, KL Div: 381.6578
Epoch[15/15], Step [2840/3750], Reconst Loss: 1358.7931, KL Div: 418.6161
Epoch[15/15], Step [2850/3750], Reconst Loss: 1471.1628, KL Div: 382.0204
Epoch[15/15], Step [2860/3750], Reconst Loss: 1343.5646, KL Div: 420.7805
Epoch[15/15], Step [2870/3750], Reconst Loss: 1149.5374, KL Div: 379.3694
Epoch[15/15], Step [2880/3750], Reconst Loss: 1350.4497, KL Div: 413.4706
Epoch[15/15], Step [2890/3750], Reconst Loss: 1221.2340, KL Div: 391.4824
Epoch[15/15], Step [2900/3750], Reconst Loss: 1296.9874, KL Div: 410.8819
Epoch[15/15], Step [2910/3750], Recons